<a href="https://colab.research.google.com/github/ImagingDataCommons/CloudSegmentator/blob/v1.3.1/workflows/TotalSegmentator/Notebooks/uploadPerFrameToBigquery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **This Notebook decompresses the csv.lz4 files containing flattened perframefunctionalgroupssequence attribute and pushes the data to biqquery table**

### **Installing Packages**

In [ ]:
%%capture
!apt-get install lz4

### **Importing Packages**

In [ ]:
from google.colab import auth
from google.cloud import bigquery
import ast
import pandas as pd
auth.authenticate_user()

In [ ]:
project_id='my project'
client = bigquery.Client(project=project_id)

### **Read the Terra data table with links to output generated from the [perframefunctionalgroupsequence workflow](https://dockstore.org/workflows/github.com/ImagingDataCommons/CloudSegmentator/perFrameFunctionalGroupSequenceExtractionOnTerra:main)**

In [ ]:
pffgs_terra_data_table=pd.read_table('path to tsv file with output links')
pffgs_terra_data_table

### **Decompress the csv.lz4, Read csv and load data to bigquery**

In [ ]:
csv_files= pffgs_terra_data_table.csvFile_PerFrameFunctionalGroupsSequence.to_list()
csv_files

### **Choose a bigquery table**

In [ ]:
bq_table_name="project.dataset.table"

In [ ]:
perframe_job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("FrameContentSequence_DimensionIndexValues", bigquery.enums.SqlTypeNames.INT64, mode="REPEATED"),
    bigquery.SchemaField("PlanePositionSequence_ImagePositionPatient", bigquery.enums.SqlTypeNames.FLOAT64, mode="REPEATED")
])


In [ ]:
for csv_file in csv_files:
  !rm perFrameFunctionalGroupSequence.csv
  !gsutil cp $csv_file .
  !lz4 -d --rm perFrameFunctionalGroupSequence.csv.lz4
  perframe_df= pd.read_csv('perFrameFunctionalGroupSequence.csv')
  # Convert string representation of a list to a list
  perframe_df['FrameContentSequence_DimensionIndexValues'] = perframe_df['FrameContentSequence_DimensionIndexValues'].apply(ast.literal_eval)

  # Now convert the list of strings to a list of integers
  perframe_df['FrameContentSequence_DimensionIndexValues'] = perframe_df['FrameContentSequence_DimensionIndexValues'].apply(lambda x: [int(i) for i in x])

  # Do the same for PlanePositionSequence_ImagePositionPatient
  perframe_df['PlanePositionSequence_ImagePositionPatient'] = perframe_df['PlanePositionSequence_ImagePositionPatient'].apply(ast.literal_eval)
  perframe_df['PlanePositionSequence_ImagePositionPatient'] = perframe_df['PlanePositionSequence_ImagePositionPatient'].apply(lambda x: [float(i) for i in x])

  # Now load the dataframe into BigQuery
  client.load_table_from_dataframe(perframe_df, bq_table_name, job_config=perframe_job_config)

